# Image embedding feature extraction & Image comparison

### [ 기능적 요구사항 ] 
1. emdedding 기능
   1. CPU에서 동작 가능
   2. 이미지 사이즈 관계 없이 동작
   3. embedding vector의 차원 수 제한x but 유사도 검색 시 성능 저하 발생 x, 유사도 분류될 수 있을 수준 
2. 유사도 검색
   1. vector database 활용해 이미지 유사도 검색 가능해야 함
      - cosine similarity
   2. 같은 그룹의 이미지끼리 유사도 높고, 다른 그룹 이미지 유사도 낮아 -> 분류기의 기능 동작  
3. 시각화 위한 2D projection
   - 고차원의 vector dataset을 시각화하여 확인 위해 2D projection 지원

### [ 비기능적 요구사항 ]
1. 성능
   1. embedding
      1. 이미지 크기 별 임베딩 시간 분석 (1024x1024 기준 50msec 이하)
      2. 임베딩 벡터의 차원 수 제한 x but 유사도 검색 시 성능 저하 발생 x, 유사도 분류될 수 있을 수준   
         - dimension에 따른 유사도 검색 시간, 검색 정확도 분석
   2. 유사도
      1. dataset 10,000, 100,000 각각의 데이터셋 사이즈에 따른 유사도 검색 시간 분석
2. 보안
3. 가용성
   1. 임베딩, 유사도 검색 진행으로 인해 다른 API 요청이 blocking 되어선 x
4. 사용성
   1. 개발 언어 상관x but node.js, go 환경에서 호출이 가능해야 함
   2. Docker 환경에서 구동되어야 함

### [데이터 요구사항]
1. open source database 사용할 것

### [시스템 제약 사항]
1. 하드웨어 제약 : GPU 사용할 수 x

### [품질 요구사항]
1. 코드 품질
2. 테스트 전략
   1. 기능별 unit test case 확보 -> code coverage 60% 이상 확보
3. 유지보수성
   1. 유지 보수 가능한 코드 품질
   2. 의미 있는 변수명, 함수명
   3. 하나의 함수 길이 - 스크롤 없이 확인 가능
   4. 추상화된 클래스 상속받는 형태로 기능 구현 - 추후 확장성 고려

## 프로젝트 일정 및 일정 계획
1. 프로젝트 일정 : 2024년 6월 3일 ~ 2024년 7월 31일
2. 개발 단계 : 초기 한 달간 - embedding 알고리즘 구현 및 검증, 이후 한달간 - 서비스에 적용 및 테스트
3. 테스트 단계 : 개발 단계에 포함
4. 유지보수 단계 : 기존 팀원들이 이어서 진행

### 참조 
[ Embedding ](https://www.syncly.kr/blog/what-is-embedding-and-how-to-use)

[ Scatter Filter ](https://ko.docs.superb-ai.com/docs/scatter-filter)

[ Auto-Curate ](https://ko.docs.superb-ai.com/docs/auto-curate)